# Self-Replicating Robot System Configurations

|ID|Design Option|Characteristics|
|-|-|-|
|1|Centralized homogeneous (CHO)|One robot is responsible for both printing components and assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|2|Decentralized homogeneous (DHO)| All robots have the capability to print components, assemble them, and gather resources or complete other objectives.|
|3|Hierarchical homogeneous (HHO)|There are a variable number of robots capable of printing components and assembling them. There are also a variable number of normal type robots.|
|4|Centralized heterogeneous (CHE)|One robot is responsible for printing components, and another (distinct) robot is responsible for assembling them. Constructed robots are of the normal type and either gather resources or complete other objectives.|
|5|Decentralized heterogeneous (DHE)|Robots have either the capability to print components or the capability to assemble them. All robots can gather resources or complete other objectives.|
|6|Hierarchical heterogeneous (CHE)|There are a variable number of robots capable of printing components, a variable number capable of assembling them (distinct from the printing group), and a variable number of normal type robots. All robots can gather resources or complete other objectives.|



# Libraries 

In [2]:
import platform
print(platform.python_version())

3.9.14


In [3]:
import random
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import sys
from matplotlib.patches import Rectangle
import os

from _plotly_future_ import v4_subplots
import plotly
import plotly.io as pio
import numpy as np
import chart_studio
import chart_studio.plotly as cspy
import chart_studio.tools as tls

from plotly.subplots import make_subplots
import plotly.graph_objects as go


import plotly.express as px

username = 'vishalgattani' # your username\n",
api_key = 'WSy2EFPTbxYYm3Rmcx53' # your api key - go to profile > settings > regenerate key\n",
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)


# Global Variables

In [4]:


# set random number generator
random.seed()

# global variables
rid = 1
nid = 0
aid = 0
pid = 0
decimalPlaces = 3

# simulation parameters
mc = False
mc_num_steps = 500
numTasksBeforeFail = 5
timesteps = 100				#; % Number of iterations/time-steps that the simulation goes through.
NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
BaseCost_NonPr = 1 			#; % Base robot cost of nonprintable components.
PrintCost_NonPr = 1 		#; % Print capability cost of nonprintable components.
AssembleCost_NonPr = 1 		#; % Assemble capability cost of nonprintable components.
BaseCost_Pr = 2 			#; % Base robot cost of printable components.
PrintCost_Pr = 2 			#; % Print capability cost of printable components.
AssembleCost_Pr = 2 		#; % Assemble capability cost of printable components.
BaseCost_Time = 2 			#; % Base robot cost of build time (in time-steps).
PrintCost_Time = 2 			#; % Print capability cost of build time (in time-steps).
AssembleCost_Time = 2 		#; % Assemble capability cost of build time (in time-steps).
Print_Efficiency = 1.0 		#; % Factor that scales raw printing materials to printable components.
Print_Amount = 1.0 			#; % Amount of raw materials converted per print task.
Collect_Amount = 1.0 		#; % Raw printing materials per collecting robot per timestep.
QualityThreshold = 0.5 		#; % Robots with a quality below this are non-functional.
Quality_incr_Chance = 5.0 	#; % Chance that a new robot’s build quality will increase.
Quality_incr_Lower = 0.01 	#; % Lower bound for quality increase amount.
Quality_incr_Upper = 0.05 	#; % Upper bound for quality increase amount.
Quality_decr_Chance = 50.0 	#; % Chance that a new robot s build quality will decrease.
Quality_decr_Lower = 0.01 	#; % Lower bound for quality decrease amount.
Quality_decr_Upper = 0.25 	#; % Upper bound for quality decrease amount.
RiskAmount_Collect = 1.0 	#; % Risk chance for the collect task type.
RiskAmount_Assemble = 0.1 	#; % Risk chance for the assemble task type.
RiskAmount_Print = 0.1 		#; % Risk chance for the print task type.
RiskQuality_Modifier = 1.0 	#; % Multiplier for impact of quality defects on risk amount.
RiskFactory_Modifier = 0.2 	#; % Multiplier for impact of factory-made robots on risk amount
RiskThreshold = 3.0
# [replicator,normal,assembler,printer]
cost_Pr = [6,2,4,4]			#; % Total cost printable
cost_NonPr = [3,1,2,2]		#; % Total cost nonprintable

timecost_base = 2			#; % time cost basic
timecost_print = 2			#; % time cost print capability
timecost_assemble = 2		#; % time cost assemble capability

timecost_normal = timecost_base
timecost_replicator = timecost_base+timecost_assemble+timecost_print
timecost_printer = timecost_base+timecost_print
timecost_assembler = timecost_base+timecost_assemble

timecost_repair_base = 1
timecost_repair_normal = timecost_repair_base
timecost_repair_replicator = timecost_repair_base
timecost_repair_printer = timecost_repair_base
timecost_repair_assembler = timecost_repair_base

table_columns = ["Time","NonPr","Printable","Materials","Env_Materials",
		"#Replicator","#Normal","#Assembler","#Printer",
		"#Assembling","#Printing","#Collecting","#Idle","#Repair",
		"#In","#Out",
		"Average Build Quality in-service","Average Build Quality of System",
		"#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
		"Environment Exhaust Time", "Printable Exhaust Time", 
		"NonPr Exhaust Time","Material Exhaust Time","Average Risk"]


# table_columns = ["Time","NonPr","Printable","Materials","Env_Materials",
# 		"#Replicator","#Normal","#Assembler","#Printer",
# 		"#Assembling","#Printing","#Collecting","#Idle",
# 		"#In","#Out",
# 		"Average Build Quality in-service","Average Build Quality of System",
# 		"#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
# 		"Environment Exhaust Time", "Printable Exhaust Time", 
# 		"NonPr Exhaust Time","Material Exhaust Time"]


def resetGlobal(t,r,n,a,p):
    global rid,nid,aid,pid,Num_Steps,NonPr,Printable,Materials,Env_Materials
    
    # global variables
    rid = r
    nid = n
    aid = a
    pid = p

    # simulation parameters
    Num_Steps = 100				#; % Number of iterations/time-steps that the simulation goes through.
    NonPr = 300.0 				#; % The robot system=s starting quantity of nonprintable components.
    Printable = 100.0 			#; % The robot system’s starting quantity of printable components.
    Materials = 50.0 			#; % The robot system’s starting quantity of raw printing materials.
    Env_Materials = 500.0 		#; % The environment’s quantity of collectable raw printing materials.
    timesteps = t


# Object::Robot

In [52]:
# robot object
class Robot:
    def __init__(self,type,build_qual,id):
        self.time = 0
        self.type = type
        self.current_task = "idle"
        self.prev_task = "idle"
        self.next_task = ""
        self.id = self.type[0]+str(id)
        self.build_qual = round(build_qual,decimalPlaces)
        self.factory_made = True
        self.tasks_dur = 0
        self.taskindex = 0
        self.previouslyBuilt = ""
        self.prevTaskDur = 0
        self.curr_repair_task_dur = 0
        self.prev_repair_task_dur = 0
        self.factory = False
        self.riskAmount = 0
        self.riskSet = False
        self.numTasksSuccess = 0
        self.numTasksFailed = 0
        self.numTasksPerformed = 0
        self.downTime = 0
        self.failureTimes = [0]
        self.operationalTimes = [0]
        self.operationalTime = 0
        self.MTBF = 0
        self.MTTR = 0
        self.MDT = 0
        self.Aoss = 0
        self.numRepairs = 0
        self.beingBuilt = ""
        self.numTasksBeforeFailure = 2
        self.numTasksRemainingBeforeFailure = 2
#         self.upTime = 1
#         self.failTime = 5
        
        if(self.type == "Replicator"):
            self.tasks = ["Assemble","Print","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Normal"):
            self.tasks = ["Collect","Repairing"]
        if(self.type == "Assembler"):
            self.tasks = ["Assemble","Collect","Repair"]
            self.beingbuiltlist = []
        if(self.type == "Printer"):
            self.tasks = ["Print","Collect","Repair"]
        self.num_tasks = len(self.tasks)
        
#     def getFailTime(self):
#         return self.failTime

    def reduceNumTasksRemainingBeforeFailure(self):
        self.numTasksRemainingBeforeFailure=self.numTasksRemainingBeforeFailure-1
    def resetNumTasksRemainingBeforeFailure(self):
        self.numTasksRemainingBeforeFailure=self.numTasksBeforeFailure
    def getNumTasksBeforeFailure(self):
        return self.numTasksBeforeFailure
    def getNumTasksRemainingBeforeFailure(self):
        return self.numTasksRemainingBeforeFailure
    def setNumTasksRemainingBeforeFailure(self,val):
        self.numTasksRemainingBeforeFailure = val
    
    def set_being_built(self,robottype):
        self.beingBuilt = robottype
    def get_being_built(self):
        return self.beingBuilt
    def set_previously_built(self, val):
        self.previouslyBuilt = val
    def get_previously_built(self):
        return self.previouslyBuilt
    def setNumTasksPerformed(self):
        self.numTasksPerformed = self.numTasksFailed+self.numTasksSuccess
    def getNumTasksPerformed(self):
        return self.numTasksPerformed
    
    def computeMTBF(self):
        try:
            if(self.operationalTime==0 and self.numTasksFailed>0):
                self.MTBF = np.nan
            else:
                self.MTBF = self.operationalTime/self.numTasksFailed
        except:
            if self.getNumTasksPerformed() == 0:
                self.MTBF = np.nan
            else:
                self.MTBF = self.operationalTime
            
    def computeMTTR(self):
        try:
            self.MTTR = self.downTime/self.numRepairs
        except:
            if self.getNumTasksPerformed() == 0:
                self.MTTR = np.nan
            elif(self.downTime == 0 or self.numRepairs==0):
                self.MTTR = self.getMTBF()
    def computeMDT(self):
        try:
            self.MDT = self.downTime/self.numTasksFailed
        except:
            if self.getNumTasksPerformed() == 0:
                self.MDT = np.nan
            elif(self.downTime == 0):
                self.MDT = math.inf
            
    def computeAoss(self):
        try:
            if(self.numTasksFailed == 0 and self.downTime == 0):
                self.Aoss =  self.MTBF/(self.MTBF+0)
            else:
                self.Aoss =  self.MTBF/(self.MTBF+self.MDT)
        except:
            if self.getNumTasksPerformed() == 0:
                self.Aoss = np.nan
            
            
    def computeRAM(self):
        self.setNumTasksPerformed()
        self.computeMTBF()
        self.computeMTTR()
        self.computeMDT()
        self.computeAoss()
        
    def printRAM(self):
        return (f'{self.id} - MTBF:{self.MTBF:.2f}, MTTR:{self.MTTR:.2f}, MDT:{self.MDT:.2f}, Aoss:{self.Aoss}')
        
    def getMTBF(self):
        return self.MTBF 
    def getMTTR(self):
        return self.MTTR 
    def getMDT(self):
        return self.MDT 
    def getAoss(self):
        return self.Aoss 
    
    def addNumRepairs(self):
        self.numRepairs += 1
    
    def taskFail(self):
        self.numTasksFailed += 1
    
    def taskSuccess(self):
        self.numTasksSuccess += 1
        
    def gettaskFail(self):
        return self.numTasksFailed
    
    def gettaskSuccess(self):
        return self.numTasksSuccess
    
    def get_type(self):
        return self.type

    def __str__(self):
#         return str(self.id)+" "+str(self.current_task)+" "+str(self.riskAmount)+",F:"+str(self.numTasksFailed)+",P:"+str(self.numTasksPerformed)
        try:
            return str(f'{self.id},{self.current_task},{self.tasks_dur},tp:{self.gettaskSuccess()}')
#             return str(self.id)+" "+str(self.current_task)+" "+str(self.get_task_dur())+" "+str(self.getPreviouslyBuilt())+" "+str(self.getbeingBuilt())
        except:
            return None
        
    def set_curr_task(self,tasktype):
        self.current_task = tasktype
        if(self.current_task == "idle"):
            self.tasks_dur = 0
        if(self.current_task == "collecting"):
            self.tasks_dur = 1
        if(self.current_task == "assembling"):
            self.tasks_dur = 2
        if(self.current_task == "printing"):
            self.tasks_dur = 2
        if(self.current_task == "repair"):
            self.failureTimes.append(self.getRobotTime())
            if(self.prev_task == "repair"):
                self.curr_repair_task_dur = self.get_task_dur()
                self.prev_repair_task_dur = self.curr_repair_task_dur
                self.tasks_dur = self.get_prev_task_dur() + 1
            else:
                self.tasks_dur = self.get_prev_task_dur() 
            self.curr_repair_task_dur = self.tasks_dur
            
    def addDownTime(self,duration):
        self.downTime += duration
    def getDownTime(self):
        return self.downTime
    def addUpTime(self):
        self.upTime += 1
    def getUpTime(self):
        return self.upTime
    def resetUpTime(self):
        self.upTime = 0
    def addOperationalTime(self,duration):
        self.operationalTime += duration
    def getOperationalTime(self):
        return self.operationalTime
        
    def setFactory(self):
        self.factory = True
    def setRobotTime(self,time):
        self.time = time
    def getRobotTime(self):
        return self.time
    # methods of robot object
    def set_prev_task(self,tasktype):
        self.prev_task = tasktype
    def set_next_task(self,tasktype):
        self.next_task = tasktype
    def set_task_dur(self,task_dur):
        self.tasks_dur = task_dur
    def get_task_dur(self):
        return self.tasks_dur
    def get_robot_id(self):
        return self.id
    def get_buid_qual(self):
        return self.build_qual
    def get_curr_task(self):
        return self.current_task
    def get_prev_task(self):
        return self.prev_task
    def get_task_dur(self):
        return self.tasks_dur
    def get_prev_task_dur(self):
        if(self.get_prev_task() == "idle"):
            return 0
        if(self.get_prev_task() == "collecting"):
            return 1
        if(self.get_prev_task() == "assembling"):
            return 2
        if(self.get_prev_task() == "printing"):
            return 2
        if(self.get_prev_task() == "repair"):
            return self.prev_repair_task_dur 
#             return 2

    
    def setRiskAmount(self,risk):
        self.riskAmount = risk
        if risk>0.0:
            self.riskSet = True
        else:
            self.riskSet = False
    def getRiskAmount(self):
        return self.riskAmount
    def getRisk(self):
        return self.riskSet


# Function::Configuration Handler

In [53]:
def configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat):
        n_replicator = 0
        n_normal = 0
        n_assembler = 0
        n_printer = 0

        c_flag = 0
        p_flag = 0
        a_flag = 0
        i_flag = 0
        repair_flag = 0

        useless_c_flag = 0
        useless_p_flag = 0
        useless_a_flag = 0
        useless_r_flag = 0

        tot_build_qual_inservice = 0
        tot_build_qual_inoutservice = 0

        build_quality_list = []

        for i in totlist:
#             print(i.get_buid_qual())
            if(i.get_buid_qual()>=0.5):
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
                tot_build_qual_inservice = tot_build_qual_inservice + i.get_buid_qual()
            else:
                tot_build_qual_inoutservice = tot_build_qual_inoutservice + i.get_buid_qual()
            build_quality_list.append(i.get_buid_qual())
        
        avg_risk = 0
        for i in robotlist:
            if i.get_curr_task()=="collecting":
                c_flag+=1
            elif i.get_curr_task()=="printing":
                p_flag+=1
            elif i.get_curr_task()=="assembling":
                a_flag+=1
            elif i.get_curr_task()=="idle":
                # print(i)
                i_flag+=1
            elif i.get_curr_task()=="repair":
                # print(i)
                repair_flag+=1

            if(checkType(i,"Replicator")):
                n_replicator += 1;
            elif(checkType(i, "Normal")):
                n_normal += 1;
            elif(checkType(i, "Assembler")):
                n_assembler += 1;
            elif(checkType(i, "Printer")):
                n_printer += 1;
#             print(i,i.factory)
                
            avg_risk += i.getRisk()
        avg_risk = avg_risk/len(robotlist)
        
        for i in useless:
            if(checkType(i,"Replicator")):
                useless_r_flag += 1;
            elif(checkType(i, "Normal")):
                useless_c_flag += 1;
            elif(checkType(i, "Assembler")):
                useless_a_flag += 1;
            elif(checkType(i, "Printer")):
                useless_p_flag += 1;



        avg_build_qual_inservice = round(tot_build_qual_inservice/len(robotlist),decimalPlaces)
        avg_build_qual_inoutservice = round(tot_build_qual_inoutservice/len(totlist),decimalPlaces)
        
        neatPrint = False
        if neatPrint:
            print("="*50)
            print(t,":\t\t",len(robotlist),[NonPr,Printable,Materials,Env_Materials])			
            print("Time\t\t",t)
            print("#Replicator:\t",n_replicator)
            print("#Normal:\t",n_normal)
            print("#Assembling:\t",n_assembler)
            print("#Printing:\t",n_printer)
            print("#Robots\t\t",len(robotlist))
            print("Materials\t",[NonPr,Printable,Materials,Env_Materials])
            print("#Assembling:\t",a_flag)
            print("#Printing:\t",p_flag)
            print("#Collecting:\t",c_flag)
            print("#Idle:\t\t",i_flag)
            print("#Repair:\t\t",repair_flag)
            
            
        
            
        ids=[]
        for j in totlist:
            isWaste = False
            if j.build_qual<=QualityThreshold:
                isWaste = True
            ids.append(j.id)

        if (Env_Materials == 0 and checkENV == 0):
            checkENV = t
        if (Printable == 0 and checkPrint == 0):
            checkPrint = t
        if (NonPr == 0 and checkNonPr == 0):
            checkNonPr = t
        if (Materials <= 1 and checkMat == 0):
            checkMat = t
            
# 		"#Replicator","#Normal","#Assembler","#Printer",
# 		"#Assembling","#Printing","#Collecting","#Idle",
# 		"#In","#Out",
# 		"Average Build Quality in-service","Average Build Quality of System",
# 		"#WasteReplicator","#WasteNormal","#WasteAssembler","#WastePrinter",
# 		"Environment Exhaust Time", "Printable Exhaust Time", 
# 		"NonPr Exhaust Time","Material Exhaust Time","Average Risk of tasks being performed"]

        
        return build_quality_list,[n_replicator,n_normal,n_assembler,n_printer,
        a_flag,p_flag,c_flag,i_flag,repair_flag,
        len(robotlist),len(useless),
        avg_build_qual_inservice,avg_build_qual_inoutservice,
        useless_r_flag,useless_c_flag,useless_a_flag,useless_p_flag,
        checkENV,checkPrint,checkNonPr,checkMat,avg_risk]

# Functions::Tasks

In [54]:
# print current resources
def printResources():
    print(NonPr,Printable,Materials,Env_Materials)

# check if robot can collect from Env_Materials
def collectCheck(robot):
    global Materials, Env_Materials, Collect_Amount
    if (Env_Materials - Collect_Amount >= 0):
        robot.setRiskAmount(taskRisk(robot))
        return True
    else:
        robot.setRiskAmount(0)
        return False

# task function - collecting
def collecting(robot):
    global Materials, Env_Materials, Collect_Amount, RiskThreshold,mc,numTasksBeforeFail
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("collecting")
    robot.set_task_dur(1)
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)     
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            Materials = Materials + Collect_Amount
            Env_Materials = Env_Materials - Collect_Amount
            robot.taskSuccess()
            robot.reduceNumTasksRemainingBeforeFailure()
        
        
    if mc:
        if robot.getRisk() == False:
            robot.setRiskAmount(taskRisk(robot))
        elif robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)
        
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            Materials = Materials + Collect_Amount
            Env_Materials = Env_Materials - Collect_Amount
            robot.taskSuccess()
    

# build robot task - assembler and replicator
# assemble task

def assembleCheck(robot,tobuild):
    # global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper
    
    if(tobuild == "Replicator"):
        i=0
        # rid = rid+1
        # robotid = rid
    if(tobuild == "Normal"):
        i=1
        # nid = nid+1
        # robotid = nid
    if(tobuild == "Assembler"):
        i=2
        # aid = aid+1
        # robotid = aid
    if(tobuild == "Printer"):
        i=3
        # pid = pid+1
        # robotid = pid

    if Printable - cost_Pr[i] >= 0 and NonPr - cost_NonPr[i] >= 0:
        if(robot.getRisk() == False):
            robot.setRiskAmount(taskRisk(robot))
        return True
    else:
        robot.setRiskAmount(0)
        return False


def assembling(robot,tobuild):
    global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper,RiskThreshold,mc,numTasksBeforeFail
    
    if(tobuild == "Replicator"):
        i=0
        taskDur = timecost_replicator
    if(tobuild == "Normal"):
        i=1
        taskDur = timecost_normal
    if(tobuild == "Assembler"):
        i=2
        taskDur = timecost_assembler
    if(tobuild == "Printer"):
        i=3
        taskDur = timecost_printer

    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("assembling")
    robot.set_task_dur(taskDur)
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            robot.taskSuccess()
            if(robot.type=="Assembler" or robot.type=="Replicator"):
                if(tobuild == "Replicator"):
                    i=0
                    rid = rid+1
                    robotid = rid
                if(tobuild == "Normal"):
                    i=1
                    nid = nid+1
                    robotid = nid
                if(tobuild == "Assembler"):
                    i=2
                    aid = aid+1
                    robotid = aid
                if(tobuild == "Printer"):
                    i=3
                    pid = pid+1
                    robotid = pid

                # subtract resources
                Printable = Printable - cost_Pr[i]
                NonPr = NonPr - cost_NonPr[i]

                robot.beingbuiltlist.append(tobuild[0]+str(robotid))
                robot.reduceNumTasksRemainingBeforeFailure()
                return True
        else:
            robot.set_prev_task(robot.get_curr_task())
            robot.set_curr_task("idle")
            robot.set_task_dur(0)
            robot.setRiskAmount(0)
            return False
    if mc:
        if robot.getRisk() == True and robot.get_prev_task()=="assembling":
            robot.setRiskAmount(taskRisk(robot))
        if robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            if(robot.getRisk() == True and robot.get_task_dur() - 1 == 0):
                robot.setRiskAmount(0)

            if(robot.type=="Assembler" or robot.type=="Replicator"):
                if(tobuild == "Replicator"):
                    i=0
                    rid = rid+1
                    robotid = rid
                if(tobuild == "Normal"):
                    i=1
                    nid = nid+1
                    robotid = nid
                if(tobuild == "Assembler"):
                    i=2
                    aid = aid+1
                    robotid = aid
                if(tobuild == "Printer"):
                    i=3
                    pid = pid+1
                    robotid = pid

                # subtract resources
                Printable = Printable - cost_Pr[i]
                NonPr = NonPr - cost_NonPr[i]

                robot.beingbuiltlist.append(tobuild[0]+str(robotid))
                return True
        else:
            robot.set_prev_task(robot.get_curr_task())
            robot.set_curr_task("idle")
            robot.set_task_dur(0)
            robot.setRiskAmount(0)
            return False

def assemble(builder,tobuild):
    global rid,nid,aid,pid,Printable,NonPr,Quality_incr_Chance,Quality_incr_Lower, Quality_incr_Upper

    if(builder.type=="Assembler" or builder.type=="Replicator"):
        if(tobuild == "Replicator"):
            i=0

        if(tobuild == "Normal"):
            i=1

        if(tobuild == "Assembler"):
            i=2

        if(tobuild == "Printer"):
            i=3

        AssemblerQuality = builder.get_buid_qual()
        # robot's build quality
        rand = round(random.uniform(0,1),decimalPlaces)
        if rand > round((1.0 - Quality_incr_Chance/100),decimalPlaces):
            RobotQuality = AssemblerQuality + random.uniform(Quality_incr_Lower, Quality_incr_Upper)
        elif rand < Quality_decr_Chance :
            RobotQuality = AssemblerQuality - random.uniform(Quality_decr_Lower, Quality_decr_Upper)
        else :
            RobotQuality = AssemblerQuality
        # print(builder,builder.beingbuiltlist)
        newRobot = Robot(tobuild,RobotQuality,builder.beingbuiltlist.pop(0)[1:])
        builder.taskSuccess()
        return newRobot
    else:
        return None


def printCheck(robot):
    if(robot.type=="Replicator" or robot.type=="Printer"):
        global Print_Efficiency, Print_Amount, Materials, Printable
        if Materials - (Print_Efficiency*Print_Amount) > 0:
#             if(robot.getRisk() == False):
#                 robot.setRiskAmount(taskRisk(robot))
            return True
        else:
            return False
    else:
        return False

def printing(robot):
    global Print_Efficiency, Print_Amount, Materials, Printable,mc,numTasksBeforeFail
    robot.set_prev_task(robot.get_curr_task())
    robot.set_curr_task("printing")
    robot.set_task_dur(PrintCost_Time)
    
    flag = 1
    if not mc:
        if robot.getNumTasksRemainingBeforeFailure()==0:
            robot.taskFail()
            flag = 0
            repairing(robot)
        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            robot.taskSuccess()
            Materials = Materials - (Print_Efficiency*Print_Amount)
            Printable = Printable + (Print_Efficiency*Print_Amount)
            robot.reduceNumTasksRemainingBeforeFailure()
    if mc:
        if robot.getRisk() == False:
            robot.setRiskAmount(taskRisk(robot))
        if robot.getRiskAmount() > RiskThreshold:
            robot.taskFail()
            flag = 0
            repairing(robot)

        if flag:
            robot.addOperationalTime(robot.get_task_dur())
            if(robot.getRisk() == True and robot.get_task_dur() - 1 == 0):
                robot.taskSuccess()
                robot.setRiskAmount(0)


            Materials = Materials - (Print_Efficiency*Print_Amount)
            Printable = Printable + (Print_Efficiency*Print_Amount)

def repairing(robot):
    global mc
    robot.set_next_task(robot.get_curr_task())
    robot.set_prev_task(robot.get_curr_task())
    repair_task_dur = robot.get_task_dur() # + basecost to repair? 
    robot.set_curr_task("repair")
    robot.addNumRepairs()
    robot.set_task_dur(repair_task_dur)
    robot.addDownTime(repair_task_dur)
    if not mc:
        robot.resetNumTasksRemainingBeforeFailure()

def resetTasks(robot):
    robot.set_prev_task(robot.get_curr_task())
    robot.set_task_dur(0)
    robot.set_curr_task("idle")

def checkCurrentTask(robot,current_task):
    return robot.get_curr_task() == current_task
def checkPreviousTask(robot,previous_task):
    return robot.get_prev_task() == previous_task

def checkType(robot,robot_type):
    return robot.get_type() == robot_type



# Task Risk

RiskTask = RiskTask_Type + (1.0 − RobotQuality) ∗ RiskTask_Type ∗ RiskQuality_Modifier

if Robot ∈ FactoryMade : RiskTask = RiskTask ∗ RiskFactory_Modifier

if robot is factory made
riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskFactory_Modifier)

if robot is not factor made
riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskQuality_Modifier)

In [55]:
def taskRisk(robot):
    rand = round(random.uniform(0,1),decimalPlaces)
    if robot.factory == True:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0
        elif(currTask == "collecting"):
            RiskTask_Type = 1
        elif(currTask == "assembling"):
            RiskTask_Type = 1
        elif(currTask == "printing"):
            RiskTask_Type = 1
        elif(currTask == "repair"):
            RiskTask_Type = 0
#         riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * rand * RiskFactory_Modifier 
        riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskFactory_Modifier)
    else:
        currTask = robot.get_curr_task()
        if(currTask == "idle"):
            RiskTask_Type = 0            
        elif(currTask == "collecting"):
            RiskTask_Type = 1
        elif(currTask == "assembling"):
            RiskTask_Type = 1
        elif(currTask == "printing"):
            RiskTask_Type = 1
        elif(currTask == "repair"):
            RiskTask_Type = 0
        
#         riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * RiskTask_Type * RiskQuality_Modifier
        riskTask = RiskTask_Type + (1.0 - robot.get_buid_qual()) * rand * RiskQuality_Modifier 
        riskTask = (1.0 - robot.get_buid_qual()) * (RiskTask_Type + rand * RiskQuality_Modifier)
#     print(robot.id,riskTask)
    return riskTask

# CHO

In [56]:
def CHO(timesteps,df,init_build_qual,printRAMmetrics=True,mc=False):
    resetGlobal(timesteps,1,0,0,0)
    robot = Robot("Replicator",init_build_qual,rid)
    robot.setFactory()
    totlist = [robot]
    robotlist = [robot]
    useless = []

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []
    
    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0

    for t in range(0,timesteps):
#         print("-"*50)
        for i in range(len(robotlist)):
            # assign risks for tasks
            robotlist[i].setRobotTime(t)
            
            # IDLE
            if(robotlist[i].current_task=="idle"):
                # Replicator
                if(robotlist[i].type == "Replicator"):
                    
                    if(assembleCheck(robotlist[i],"Normal")):
                        isAssembling = assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # Normal
                elif(robotlist[i].type == "Normal"):

                    # print(t,robotlist[i].id,canCollect)
                    if (collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                
            # REPAIR            
            elif(robotlist[i].get_curr_task()=="repair"):
                prevTask = robotlist[i].get_prev_task()
                # tasks of replicator in CHO
                if(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Replicator"):
                    if(prevTask=="assembling" and assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    elif(prevTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                # tasks of normal in CHO
                elif(robotlist[i].get_task_dur() - 1 == 0 and robotlist[i].type == "Normal"):
                    if(prevTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                else:
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)

            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].get_task_dur() - 1)
                    
                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Normal")):
                        assembling(robotlist[i],"Normal")
                    elif(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    else:
                        isAssembling = False
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    # it enters this loop only when it has to pop a new robot
                    if(robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Normal")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)
                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
              
            
#         print("Time",t,"w/",len(robotlist),"Robots") 
#         for j in robotlist:
#             print(j)
#         print("-"*50)
            
        #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    MTBFlist = []
    MTTRlist = []
    MDTlist = []
    Aosslist = []
    for robot in robotlist:
        robot.computeRAM()
        if math.isnan(robot.getAoss())==False: 
            MTBFlist.append(robot.getMTBF())
            MTTRlist.append(robot.getMTTR())
            MDTlist.append(robot.getMDT())
            Aosslist.append(robot.getAoss())

    lambda_robots = np.reciprocal(MTBFlist) 
    mu_robots = np.reciprocal(MTTRlist) 
    MDT_SRRS = sum(np.reciprocal(MDTlist))
    sigma_lambda = sum(lambda_robots)
    pi_lambda = np.prod(lambda_robots)
    sigma_mu = sum(mu_robots)
    pi_mu = np.prod(mu_robots)
    lambda_SRRS = pi_lambda*sigma_mu/pi_mu
    mu_SRRS = sigma_mu
    MTBF_SRRS = 1/lambda_SRRS
    MTTR_SRRS= 1/mu_SRRS
    Aoss_SRRS = MTBF_SRRS/(MTBF_SRRS+MDT_SRRS)
    if printRAMmetrics:
        pass
    
    # generate a dataframe with metrics
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    ram_df = pd.DataFrame(columns=["MTBF","MTTR","MDT","Aoss"],data=[[MTBF_SRRS,MTTR_SRRS,MDT_SRRS,Aoss_SRRS]])
    return df,ram_df

# Testing

In [57]:
def SRRS(config,timesteps,plot=False):
    df = pd.DataFrame(columns = table_columns)
    build_qual_range = [0.85,0.95]
    init_build_qual = random.uniform(build_qual_range[0],build_qual_range[1])
    if config=="CHO":
        df,ram_df = CHO(timesteps,df,init_build_qual)
        printSRRSConfigRAMmetrics(config,ram_df)
    if(plot):
        graphOutputs(config,df)
    return df,ram_df

def SRRSmc(config,timesteps,mc_num_steps,plot=False):
    mcSimCols = ['Time', 'NonPr', 'Printable', 'Materials', 'Env_Materials',
           '#Replicator', '#Normal', '#Assembler', '#Printer', '#Assembling',
           '#Printing', '#Collecting', '#Idle', '#Repair', '#In', '#Out',
           'Average Build Quality in-service', 'Average Build Quality of System',
           '#WasteReplicator', '#WasteNormal', '#WasteAssembler', '#WastePrinter',
           'Environment Exhaust Time', 'Printable Exhaust Time',
           'NonPr Exhaust Time', 'Material Exhaust Time', 'Average Risk',
           'Print Capacity', 'Assembling Capacity', 'Collection Capacity', 'MTBF',
           'MTTR', 'MDT', 'Aoss']
    mcdfsim = pd.DataFrame(columns=mcSimCols)
    for i in range(1,mc_num_steps+1):
        sys.stdout.write("\rRunning iteration: {0}".format(i))
        sys.stdout.flush()
        df,ram_df = SRRS("CHO",timesteps)
        df_stats = df.describe()
        avg_avgRisk = df_stats.iloc[1]["Average Risk"]
        last_row = df.tail(1)
        last_row["Average Risk"].values[:] = avg_avgRisk
        cols = df.columns
        mcdf = pd.DataFrame(columns=cols)
        mcdf = pd.concat([mcdf,last_row], ignore_index=True)
        mcdf = pd.concat([mcdf, ram_df], axis=1, join='inner')
        last_row = mcdf.tail(1)
        mcdfsim = pd.concat([mcdfsim, last_row], ignore_index=True)
#     display(mcdfsim[["Aoss"]])
    return mcdfsim
    
def printSRRSConfigRAMmetrics(config,ram_df):
    display(ram_df)

def graph_RobotTasks(config,df):
    fig = px.bar(df, x="Time", y=['#Printing','#Assembling','#Collecting', '#Idle', '#Repair'], title=config+" Robot Tasks vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show()

def graph_InOutServiceRobots(config,df):
    fig = px.bar(df, x="Time", y=['#In','#Out'], title=config+" Robots in service vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show() 
    
def graph_RobotTypes(config,df):
    fig = px.bar(df, x="Time", y=['#Replicator', '#Normal', '#Assembler', '#Printer'], title=config+" Robots Types vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig.update_layout(hovermode="x")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='#Robots')
    fig.show() 
    
def graph_Resources(config,df):
#     fig = px.bar(df, x="Time", y=['NonPr', 'Printable', 'Materials', 'Env_Materials'], title="Resources vs Time") #,labels={'#In':'#in-service','#Out':'#out-service'})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["Time"], y=df['NonPr'], mode="lines",name='NonPr'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Printable'], mode="lines",name='Printable'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Materials'], mode="lines",name='Materials'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Env_Materials'], mode="lines",name='Env_Materials'))
    fig.update_layout(hovermode="x",title=config+" Resources vs Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Resources')
    fig.show()
    
def graph_AvgBuildQuality(config,df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality in-service'], mode="lines",name='Avg. Build Quality-In Service'))
    fig.add_trace(go.Scatter(x=df["Time"], y=df['Average Build Quality of System'], mode="lines",name='Avg. Build Quality-System'))
    fig.update_layout(hovermode="x",title=config+" Robot Build Quality vs Time")
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text='Build Quality')
    fig.show()
    
def graphOutputs(config,df):
    graph_RobotTasks(config,df)
    graph_InOutServiceRobots(config,df)
    graph_RobotTypes(config,df)
    graph_Resources(config,df)
    graph_AvgBuildQuality(config,df)
    

    

In [63]:
def main():
    global RiskThreshold,QualityThreshold,mc_num_steps,timesteps,mc
    RiskThreshold=0.3
    QualityThreshold=0.5
    mc_num_steps = 20
    timesteps = 150
    mc = False
    try:
        if mc:
            mcdf = SRRSmc("CHO",timesteps,mc_num_steps,True)
            display(mcdf)
            print(mcdf.columns)
            mean_mcdf = mcdf[['Print Capacity', 'Assembling Capacity', 'Collection Capacity', 'MTBF',
       'MTTR', 'MDT', 'Aoss','#In', '#Out','Average Build Quality of System']].mean()
            display(mean_mcdf)
        else:
            df,ram_df = SRRS("CHO",timesteps,True)    
    except Exception as err:
        print(f"Unexpected {err=}, {type(err)=}")
        raise

if __name__ == "__main__":
    main()

Running iteration: 1

,MTBF,MTTR,MDT,Aoss
0,3499.236367,0.039723,24.0,0.993188


Running iteration: 2

,MTBF,MTTR,MDT,Aoss
0,351338.197714,0.037374,25.0,0.999929


Running iteration: 3

...

Running iteration: 20

,MTBF,MTTR,MDT,Aoss
0,3185.518605,0.061872,15.0,0.995313


,Time,NonPr,Printable,Materials,Env_Materials,#Replicator,#Normal,#Assembler,#Printer,#Assembling,#Printing,#Collecting,#Idle,#Repair,#In,#Out,Average Build Quality in-service,Average Build Quality of System,#WasteReplicator,#WasteNormal,#WasteAssembler,#WastePrinter,Environment Exhaust Time,Printable Exhaust Time,NonPr Exhaust Time,Material Exhaust Time,Average Risk,Print Capacity,Assembling Capacity,Collection Capacity,MTBF,MTTR,MDT,Aoss
0,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.781,0.781,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.391896,1.0,1.0,59.0,3499.236367,0.039723,24.0,0.993188
1,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.787,0.787,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.388678,1.0,1.0,59.0,351338.197714,0.037374,25.0,0.999929
2,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.721,0.721,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.429692,1.0,1.0,59.0,2.929265,0.032251,30.0,0.088956
3,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.76,0.76,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.37683,1.0,1.0,59.0,11767.364759,0.04311,22.0,0.998134
4,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.822,0.822,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.372236,1.0,1.0,59.0,433.656869,0.051154,18.0,0.960147
5,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.816,0.816,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.371742,1.0,1.0,59.0,2061.076814,0.053089,17.0,0.991819
6,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.748,0.748,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.390287,1.0,1.0,59.0,181.13431,0.03988,24.0,0.883003
7,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.794,0.794,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.377068,1.0,1.0,59.0,1262.896244,0.048991,19.0,0.985178
8,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.77,0.77,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.387298,1.0,1.0,59.0,346.884456,0.044453,21.0,0.942917
9,149.0,242.0,1.0,533.0,0.0,1.0,58.0,0.0,0.0,0.0,1.0,0.0,58.0,0.0,59.0,0.0,0.763,0.763,0.0,0.0,0.0,0.0,149.0,0.0,0.0,0.0,0.392126,1.0,1.0,59.0,2057.621647,0.04082,23.0,0.988946


Index(['Time', 'NonPr', 'Printable', 'Materials', 'Env_Materials',
       '#Replicator', '#Normal', '#Assembler', '#Printer', '#Assembling',
       '#Printing', '#Collecting', '#Idle', '#Repair', '#In', '#Out',
       'Average Build Quality in-service', 'Average Build Quality of System',
       '#WasteReplicator', '#WasteNormal', '#WasteAssembler', '#WastePrinter',
       'Environment Exhaust Time', 'Printable Exhaust Time',
       'NonPr Exhaust Time', 'Material Exhaust Time', 'Average Risk',
       'Print Capacity', 'Assembling Capacity', 'Collection Capacity', 'MTBF',
       'MTTR', 'MDT', 'Aoss'],
      dtype='object')


Print Capacity                         1.000000
Assembling Capacity                    1.000000
Collection Capacity                   59.000000
MTBF                               19427.776701
MTTR                                   0.044062
MDT                                   22.100000
Aoss                                   0.888701
#In                                   59.000000
#Out                                   0.000000
Average Build Quality of System        0.774900
dtype: float64

In [ ]:
def DHO(timesteps,df,init_build_qual):
    resetGlobal(timesteps,1,0,0,0)
    robot = Robot("Replicator",init_build_qual,rid)
    robot.setFactory()
    totlist = [robot]
    robotlist = [robot]
    useless = []
    

    # number of bots working
    listnumCollecting = []
    listnumPrinting = []
    listnumAssembling = []

    # use lists
    tcoordslist = []
    rcoordslist = []
    wastecoordslist = []
    t_build_quality_list = []

    #Lists used for visualization
    checkENV = 0
    checkPrint = 0
    checkNonPr = 0
    checkMat = 0
    
    for t in range(0,timesteps):
        print(t,"="*20)
        for i in range(len(robotlist)):
            robotlist[i].setRobotTime(t)
            print(robotlist[i])
            # IDLE
            if(robotlist[i].current_task=="idle"):

                # Replicator
                if(robotlist[i].type == "Replicator"):

                    if(assembleCheck(robotlist[i],"Replicator")):
                        assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    # print(t,robotlist[i].id,canCollect)
                    if canCollect:
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
            # REPAIR            
            elif(robotlist[i].current_task=="repair"):
                prevTask = robotlist[i].get_prev_task()
                # nextTask = robotlist[i].get_next_task()
                
                # tasks of replicator in CHO
                if(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    if(prevTask=="assembling" and assembleCheck(robotlist[i],"Replicator")):
                        assembling(robotlist[i],"Replicator")
                    elif(prevTask=="printing" and printCheck(robotlist[i])):
                        printing(robotlist[i])
                    elif(prevTask=="collecting" and collectCheck(robotlist[i])):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                
                else:
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)
                            
            # NOT IDLE
            else:
                # reduce task duration every time step
                if(robotlist[i].tasks_dur - 1 != 0):
                    robotlist[i].set_task_dur(robotlist[i].tasks_dur - 1)

                # Replicator 
                elif(robotlist[i].tasks_dur - 1 == 0 and robotlist[i].type == "Replicator"):
                    # check if it can keep assembling next time step
                    if(assembleCheck(robotlist[i],"Replicator")):
                        isAssembling = assembling(robotlist[i],"Replicator")
                    elif(printCheck(robotlist[i])):
                        isPrinting = printing(robotlist[i])	
                    elif(collectCheck(robotlist[i])):
                        collecting(robotlist[i])	
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")

                    # it enters this loop only when it has to pop a new robot
                    if(robotlist[i].get_prev_task()=="assembling"):
                        newbot = assemble(robotlist[i],"Replicator")
                        if newbot and newbot.build_qual>=QualityThreshold:
                            if(newbot.type == "Normal"):
                                canCollect = collectCheck(newbot)
                                if canCollect:
                                    collecting(newbot)
                            if(newbot.type == "Replicator"):
                                if(assembleCheck(newbot,"Replicator")):
                                    assembling(newbot,"Replicator")
                                elif(printCheck(robotlist[i])):
                                    printing(robotlist[i])
                                elif(collectCheck(robotlist[i])):
                                    collecting(robotlist[i])	
                                else:
                                    newbot.set_prev_task(robotlist[i].get_curr_task())
                                    newbot.set_task_dur(0)
                                    newbot.set_curr_task("idle")
                            totlist.append(newbot)
                            robotlist.append(newbot)
                        else:
                            totlist.append(newbot)
                            useless.append(newbot)
                        robotlist[i].set_prev_task(robotlist[i].current_task)

                # Normal
                elif(robotlist[i].type == "Normal"):
                    canCollect = collectCheck(robotlist[i])
                    if(canCollect):
                        collecting(robotlist[i])
                    else:
                        robotlist[i].set_prev_task(robotlist[i].get_curr_task())
                        robotlist[i].set_task_dur(0)
                        robotlist[i].set_curr_task("idle")
                

        #after simulation record
        build_quality_list,vals = configHandler(t,totlist,robotlist,useless,checkENV,checkPrint,checkNonPr,checkMat)
        df.loc[len(df)] = [t,NonPr,Printable,Materials,Env_Materials]+vals
        tcoordslist.append(t)
        rcoordslist.append(len(robotlist)) 
        wastecoordslist.append(len(useless))
        t_build_quality_list.append(build_quality_list)
    
    # setup RAM metrics of robots
    for robot in robotlist:
            robot.computeRAM()
    
    # generate a dataframe with metrics
    df["Print Capacity"] = df[["#Printer", "#Replicator"]].sum(axis=1)
    df["Assembling Capacity"] = df[["#Assembler", "#Replicator"]].sum(axis=1)
    df["Collection Capacity"] = df[["#Printer", "#Replicator", "#Assembler", "#Normal"]].sum(axis=1)
    return df